In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
import ipywidgets as widgets

import datetime
import yfinance as yf
import lxml
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import plotly.subplots as subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers
pio.renderers.default = "iframe"

In [ ]:
symbols = pd.read_csv('data/symbol.csv', sep=',')
symbols.fillna('', inplace=True)
ticker = None

def get_ticker(TickerName):
    global ticker
    x = symbols.loc[symbols.SYMBOL == TickerName].to_numpy()
    ticker_name = x[0][1] + '.MI'
    ticker = yf.Ticker(ticker_name)
    info = ticker.get_info()
    html = (f'<h1><a href="{info["website"]}">{info["longName"]}</a></h1>'
            f'<div><span>{info["country"]}</span><span style="margin: 0 10px 0 10px">-</span><span>{info["city"]}</span></div>'
            f'<div><span>{info["sector"]}</span><span style="margin: 0 10px 0 10px">-</span><span>{info["industry"]}</span></div>'
            f'<div style="margin-top: 10px"><p>{info["longBusinessSummary"]}</p></div>')
    display(HTML(html))
    
def plot_ticker(Period, Interval):
    df = ticker.history(period=Period, interval="1h")
    fig = subplots.make_subplots(rows=2, cols=1, shared_xaxes=True)
    fig.add_trace(go.Scatter(x=df.index, y=df.Close, name="close price"), row=1, col=1)
    fig.add_trace(go.Bar(x=df.index, y=df.Volume, name="volume"), row=2, col=1)
    fig.update_layout(title_text=ticker.ticker)
    fig.show()
    
    fig = go.Figure(data=[go.Candlestick(x=df.index, open=df.Open, high=df.High, low=df.Low, close=df.Close)])
    fig.show()
    
interact(get_ticker, TickerName=[tuple(x) for x in symbols[['NAME', 'SYMBOL']].to_numpy()])
interact(plot_ticker, Period=['1d','5d','1mo','3mo','6mo','1y','2y','5y','10y','ytd','max'], Interval=['1h', '1d', '5d'])